In [14]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm

In [15]:
transforms=torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

In [16]:
dataset=torchvision.datasets.MNIST(root= '/myprogect/fync/dataset',download=True,transform=transforms)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /myprogect/fync/dataset\MNIST\raw\train-images-idx3-ubyte.gz to /myprogect/fync/dataset\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /myprogect/fync/dataset\MNIST\raw\train-labels-idx1-ubyte.gz to /myprogect/fync/dataset\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /myprogect/fync/dataset\MNIST\raw\t10k-images-idx3-ubyte.gz to /myprogect/fync/dataset\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /myprogect/fync/dataset\MNIST\raw\t10k-labels-idx1-ubyte.gz to /myprogect/fync/dataset\MNIST\raw



In [17]:
# plt.imshow(dataset[0][0])
# plt.colorbar()

In [18]:
dataset[0][0].shape

torch.Size([1, 28, 28])

In [19]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16,shuffle=True)
for sample in dataloader:
    img, label = sample
    label=F.one_hot(label,num_classes=10)
    label = label.float()
    print(label)
    break

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])


BCHW->BC=C+H+W

In [20]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten =nn.Flatten()
        self.Linear1=nn.Linear(784,100)
        self.act=nn.Sigmoid()
        self.LInear2=nn.Linear(100,10)

    def forward(self, x):
        feature = self.flatten(x)
        feature = self.Linear1(feature)
        feature = self.act(feature)
        out = self.LInear2(feature)
        #out = self.act(feature)
        return out
    
net = SimpleNet()

In [21]:
loss_fn=nn.CrossEntropyLoss()

lr = learning rate

In [22]:
optimizer=torch.optim.SGD(net.parameters(),lr=1e-3,momentum=0.9)

In [23]:
def get_accuracy(pred,labal):
    return (pred.detach().numpy().argmax(1)==label.numpy().argmax(1)).mean()

In [26]:
epochs=8
for epoch in range(epochs):
    loss_val=0
    acc_val=0
    for sample in tqdm(dataloader):
        img,label = sample
        label=F.one_hot(label,num_classes=10)
        label = label.float()
        optimizer.zero_grad()

        pred = net (img)
        loss = loss_fn(pred, label)
        loss.backward()
        optimizer.step()
        
        loss_val+=loss.item()
        acc_val +=get_accuracy(pred,label)
    print(loss_val/len(dataloader))
    print(acc_val/len(dataloader))

100%|█████████████████████████████████████████████████████████████████████████████| 3750/3750 [00:10<00:00, 374.80it/s]


0.28503176732013624
0.9182666666666667


100%|█████████████████████████████████████████████████████████████████████████████| 3750/3750 [00:09<00:00, 406.90it/s]


0.27594043834507465
0.9211166666666667


100%|█████████████████████████████████████████████████████████████████████████████| 3750/3750 [00:08<00:00, 418.58it/s]


...

0.24598090118492644
0.9293


100%|█████████████████████████████████████████████████████████████████████████████| 3750/3750 [00:09<00:00, 385.18it/s]


0.2395509822147588
0.9312


100%|█████████████████████████████████████████████████████████████████████████████| 3750/3750 [00:09<00:00, 391.81it/s]

0.23344695678104957
0.9329666666666667


In [25]:
torch.save(net.state_dict(),'mymodel.pth')

net.load_state_dict(torch.load'mymodel.pth')